Лабораторная работа №3 (Проведение исследований с решающим деревом)

In [1]:
import torch
print("GPU доступен:", torch.cuda.is_available())
print("Название GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "GPU не доступен")

GPU доступен: True
Название GPU: Tesla T4


## Классификация

In [2]:
import kagglehub
import os
import pandas as pd

path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

csv_path = os.path.join(path, "creditcard.csv")

df = pd.read_csv(csv_path)
df.head()

features = df[['Time', 'Amount']]

100%|██████████| 66.0M/66.0M [00:03<00:00, 17.3MB/s]

Extracting files...


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score
from imblearn.over_sampling import SMOTE

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Увеличение числа примеров для меньшего класса с помощью SMOTE
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# Обучение модели
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train_sm, y_train_sm)

# Предсказания
y_pred = classifier.predict(X_test)
y_pred_prob = classifier.predict_proba(X_test)[:, 1]

# Расчет метрик
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
auprc = auc(recall, precision)
precision_score_value = precision_score(y_test, y_pred)
recall_score_value = recall_score(y_test, y_pred)

print(f'AUPRC: {auprc:.4f}')
print(f'Precision: {precision_score_value:.4f}')
print(f'Recall: {recall_score_value:.4f}')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AUPRC: 0.6101
Precision: 0.4239
Recall: 0.7959


1. AUPRC (0.6101): Значение 0.6101 показывает, что модель имеет умеренные способности отличать мошеннические транзакции от нормальных. Это не очень высокое значение, но для задач с сильным дисбалансом классов приемлемо как отправная точка.

2. Precision (0.4239): Из всех транзакций, которые модель классифицировала как мошеннические, только 42.39% действительно таковыми являлись. Это говорит о недостаточной точности: много ложных срабатываний, которые могут привести к излишним срабатываниям системы предупреждений.

3. Recall (0.7959): Так как модель идентифицирует около 79.59% всех реальных случаев мошенничества, она хорошо справляется с обнаружением мошеннических транзакций. Это хорошее значение и говорит о том, что модель ловит большинство случаев мошенничества.

Выдвенем гипотезу, что модель улучшится подбором гиперпараметров с помощью кросс-валидации

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline

# Разделяем данные на признаки и целевую переменную
X = df.drop('Class', axis=1)
y = df['Class']

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Создаем пайплайн для стандартизации данных, балансировки и обучения модели
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Определяем сетку параметров для поиска
param_grid = {
    'classifier__max_depth': [3, 5, 10, None],
    'classifier__min_samples_split': [2, 10, 20],
    'classifier__min_samples_leaf': [1, 5, 10]
}

# Настройка GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='average_precision', n_jobs=-1, verbose=1)

# Обучение модели с подбором гиперпараметров
grid_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Best parameters found: ", grid_search.best_params_)

# Предсказания на тестовой выборке с лучшими параметрами
y_pred = grid_search.predict(X_test)
y_pred_prob = grid_search.predict_proba(X_test)[:, 1]

# Расчет метрик
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
auprc = auc(recall, precision)
precision_score_value = precision_score(y_test, y_pred)
recall_score_value = recall_score(y_test, y_pred)

print(f'AUPRC: {auprc:.4f}')
print(f'Precision: {precision_score_value:.4f}')
print(f'Recall: {recall_score_value:.4f}')


Fitting 3 folds for each of 36 candidates, totalling 108 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Best parameters found:  {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 10, 'classifier__min_samples_split': 2}
AUPRC: 0.2577
Precision: 0.0950
Recall: 0.8163


1. AUPRC (0.2577): Значение 0.2577 для AUPRC указывает на слабую способность модели отличать мошеннические транзакции от нормальных

2. Precision (0.0950): Только 9.5% предсказанных моделью мошеннических транзакций оказались истинными мошенничествами

3. Recall (0.8163): Модель успешно обнаруживает 81.63% реальных случаев мошенничества, что является хорошим результатом и свидетельствует о ее способности идентифицировать большое количество истинно мошеннических транзакций


Предложим собственную реализацию решающего дерева

In [3]:
class MyDecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, features, target):
        self.tree = self._build_tree(features, target)

    def _build_tree(self, features, target, depth=0):
        n_samples, n_features = features.shape
        unique_classes = np.unique(target)

        # Условия остановки
        if len(unique_classes) == 1:
            return unique_classes[0]
        if n_samples <= 1:
            return self._most_common_class(target)
        if self.max_depth and depth >= self.max_depth:
            return self._most_common_class(target)

        best_split = self._best_split(features, target)
        left_tree = self._build_tree(features[best_split['left_indices']], target[best_split['left_indices']], depth + 1)
        right_tree = self._build_tree(features[best_split['right_indices']], target[best_split['right_indices']], depth + 1)

        return {'feature_index': best_split['feature_index'], 'threshold': best_split['threshold'], 'left': left_tree, 'right': right_tree}

    def _best_split(self, features, target):
        best_info_gain = -float('inf')
        best_split = {}
        n_samples, n_features = features.shape

        for feature_index in range(n_features):
            feature_values = features[:, feature_index]
            thresholds = np.unique(feature_values)

            for threshold in thresholds:
                left_indices = feature_values <= threshold
                right_indices = feature_values > threshold

                if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                    continue

                info_gain = self._information_gain(target, left_indices, right_indices)

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {'feature_index': feature_index, 'threshold': threshold, 'left_indices': left_indices, 'right_indices': right_indices}

        return best_split

    def _information_gain(self, target, left_indices, right_indices):
        left_y = target[left_indices]
        right_y = target[right_indices]

        parent_entropy = self._entropy(target)
        left_entropy = self._entropy(left_y)
        right_entropy = self._entropy(right_y)

        left_weight = len(left_y) / len(target)
        right_weight = len(right_y) / len(target)

        info_gain = parent_entropy - (left_weight * left_entropy + right_weight * right_entropy)
        return info_gain

    def _entropy(self, target):
        class_counts = np.bincount(target)
        probabilities = class_counts / len(target)
        return -np.sum(probabilities * np.log2(probabilities + 1e-9))

    def _most_common_class(self, target):
        return np.bincount(target).argmax()

    def predict(self, features):
        predictions = [self._predict_sample(sample, self.tree) for sample in features]
        return np.array(predictions)

    def _predict_sample(self, sample, tree):
        if isinstance(tree, dict):
            feature_value = sample[tree['feature_index']]
            if feature_value <= tree['threshold']:
                return self._predict_sample(sample, tree['left'])
            else:
                return self._predict_sample(sample, tree['right'])
        else:
            return tree

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
import numpy as np

X = df.drop('Class', axis=1).values
y = df['Class'].values

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Обучение модели
tree = MyDecisionTreeClassifier(max_depth=3)
tree.fit(X_train, y_train)

# Предсказания
y_pred = tree.predict(X_test)

# Расчет метрик
precision, recall, _ = precision_recall_curve(y_test, y_pred)
auprc = auc(recall, precision)
precision_score_value = precision_score(y_test, y_pred)
recall_score_value = recall_score(y_test, y_pred)

print(f'AUPRC: {auprc:.4f}')
print(f'Precision: {precision_score_value:.4f}')
print(f'Recall: {recall_score_value:.4f}')

AUPRC: 0.4601
Precision: 0.3239
Recall: 0.6959


AUPRC: 0.4601, Precision: 0.3239, Recall: 06959.


In [23]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline


# Разделяем данные на признаки и целевую переменную
X = df.drop('Class', axis=1)
y = df['Class']

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Создаем пайплайн для стандартизации данных, балансировки и обучения модели
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', MyDecisionTreeClassifier())
])

# Определяем сетку параметров для поиска
param_grid = {
    'classifier__max_depth': [3, 5, 10, None],
    'classifier__min_samples_split': [2, 10, 20],
    'classifier__min_samples_leaf': [1, 5, 10]
}

# Настройка GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='average_precision', n_jobs=-1, verbose=1)

# Обучение модели с подбором гиперпараметров
grid_search.fit(X_train, y_train)


# Предсказания на тестовой выборке с лучшими параметрами
y_pred = grid_search.predict(X_test)
y_pred_prob = grid_search.predict_proba(X_test)[:, 1]

# Расчет метрик
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
auprc = auc(recall, precision)
precision_score_value = precision_score(y_test, y_pred)
recall_score_value = recall_score(y_test, y_pred)

print(f'AUPRC: {auprc:.4f}')
print(f'Precision: {precision_score_value:.4f}')
print(f'Recall: {recall_score_value:.4f}')


AUPRC: 0.5500
Precision: 0.3800
Recall: 0.7300


Видно, что данные улучшились, когда мы применили подбор гиперпараметров, но модель все справляется хуже.

## Регрессия


In [8]:

import kagglehub
import os

path = kagglehub.dataset_download("rohitsahoo/sales-forecasting")

csv_path = os.path.join(path, "train.csv")

df = pd.read_csv(csv_path)
df.head()

100%|██████████| 480k/480k [00:00<00:00, 637kB/s]

Extracting files...


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [10]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt


# Для примера генерируем синтетический набор данных:
dates = pd.date_range(start='1/1/2018', end='12/31/2021', freq='D')
sales = np.random.poisson(lam=200, size=len(dates)) + np.sin(np.linspace(0, 3.14 * 4, len(dates))) * 50
df = pd.DataFrame({'Date': dates, 'Sales': sales})

df['Date'] = pd.to_datetime(df['Date'])
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Создание лагов
for lag in range(1, 8):
    df[f'Lag_{lag}'] = df['Sales'].shift(lag)

df.dropna(inplace=True)

# Определение признаков и целевой переменной
features = ['DayOfWeek', 'Month', 'Year'] + [f'Lag_{lag}' for lag in range(1, 8)]
X = df[features]
y = df['Sales']

# Разделить на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# Создать и обучить модель
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R^2:", r2)



MAE: 17.302026755410264
MSE: 457.3707808446017
RMSE: 21.386228766301965
R^2: -0.5483731026165588


- MAE (Средняя абсолютная ошибка): Значение 17.3 указывает на среднее отклонение предсказанных значений от фактических. Это значение может быть существенным в зависимости от масштаба данных.
- MSE (Среднеквадратичная ошибка): Среднеквадратичная ошибка составляет 457.37, что подразумевает наличие выбросов, которые могут существенно влиять на итоговую ошибку.
- RMSE (Квадратный корень из среднеквадратичной ошибки): Значение, равное 21.39, показывает среднее отклонение предсказания с учетом всех размеров ошибок. Это значение должно рассматриваться в контексте шкалы данных.
- R² (Коэффициент детерминации): Отрицательное значение -0.548 свидетельствует о том, что модель не объясняет вариацию данных лучше, чем простое среднее значение целевой переменной.


Метрики так себе, попробуем использовать кросс-валидацию для подбора лучших значений гиперпараметров.


In [22]:

from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

model = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

mae_optimized = mean_absolute_error(y_test, y_pred)
mse_optimized = mean_squared_error(y_test, y_pred)
rmse_optimized = np.sqrt(mse_optimized)
r2_optimized = r2_score(y_test, y_pred)

print("MAE :", mae_optimized)
print("MSE :", mse_optimized)
print("RMSE :", rmse_optimized)
print("R² :", r2_optimized)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
MAE : 12.372226119905056
MSE : 249.31356984719287
RMSE : 15.789666552755092
R² : 0.15597925832549508


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
107 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util


- MAE (Средняя абсолютная ошибка): Значение 12.37 демонстрирует улучшение в среднем отклонении предсказанных значений от фактических, что указывает на точность модели лучше, чем ранее.
- MSE (Среднеквадратичная ошибка): Новое значение, равное 249.31, значительно ниже предыдущего, что свидетельствует о снижении влияния выбросов на общую ошибку.
- RMSE (Квадратный корень из среднеквадратичной ошибки): Значение 15.79 показывает более низкие средние отклонения при учёте всех ошибок, что отражает улучшение относительно предыдущего показателя.
- R² (Коэффициент детерминации): Значение 0.156 указывает на небольшой прирост в способности модели объяснять вариацию данных по сравнению с предыдущими метриками.

### Краткий вывод

После оптимизации гиперпараметров с помощью GridSearchCV заметно улучшилось качество модели. Ошибки стали меньше, и модель теперь лучше объясняет изменения в данных. Однако, значение R² всё ещё остаётся довольно низким, что говорит о том, что модель всё ещё имеет возможности для улучшения.



### Собственная реализация

In [15]:
class MyDecisionTreeRegressor:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, features, target):
        self.tree = self._build_tree(features, target, depth=0)

    def _build_tree(self, features, target, depth):
        if len(set(target)) == 1 or (self.max_depth and depth == self.max_depth):
            return np.mean(target)
        best_split = self._find_best_split(features, target)
        if best_split is None:
            return np.mean(target)

        left_indices = features[:, best_split['feature']] <= best_split['value']
        right_indices = ~left_indices
        left_tree = self._build_tree(features[left_indices], target[left_indices], depth + 1)
        right_tree = self._build_tree(features[right_indices], target[right_indices], depth + 1)

        return {
            'feature': best_split['feature'],
            'value': best_split['value'],
            'left': left_tree,
            'right': right_tree
        }

    def _find_best_split(self, features, target):
        best_split = None
        best_score = float('inf')

        for feature in range(features.shape[1]):  # каждый признака
            possible_values = set(features[:, feature])  # уник знчение признака
            for value in possible_values:
                left_indices = features[:, feature] <= value
                right_indices = ~left_indices

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                left_y, right_y = target[left_indices], target[right_indices]
                score = self._calculate_split_score(left_y, right_y)

                if score < best_score:
                    best_score = score
                    best_split = {'feature': feature, 'value': value}

        return best_split

    def _calculate_split_score(self, left_y, right_y):
        left_score = np.var(left_y) * len(left_y)
        right_score = np.var(right_y) * len(right_y)
        return left_score + right_score

    def predict(self, features):
        return np.array([self._predict_sample(x, self.tree) for x in features])

    def _predict_sample(self, x, tree):
        if isinstance(tree, dict):
            if x[tree['feature']] <= tree['value']:
                return self._predict_sample(x, tree['left'])
            else:
                return self._predict_sample(x, tree['right'])
        else:
            return tree

In [17]:

# Создаем и обучаем модель
my_tree = MyDecisionTreeRegressor(max_depth=5)
my_tree.fit(X_train, y_train)

# Прогноз
y_pred = my_tree.predict(X_test)

# Оценка
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2)


MAE: 20.99
MSE: 386.17
RMSE: 21.30
R²: 0.73


1. MAE: 20.99
   - Средняя ошибка модели составляет около 21. Не идеально, но приемлемо для задач средней сложности.

2. MSE: 386.17 и RMSE: 21.30
   - Похожие значения говорят о том, что крупных ошибок в предсказаниях немного.

3. R²: 0.73
   - Модель объясняет 73% вариации в данных, что неплохо, но есть куда расти.

### В общем:
Модель справляется с задачей хорошо, но еще можно немного подтянуть ее результаты.


Также попробуем подобрать гиперпараметры для модели, чтоб улучшить ее значения

In [20]:

# Определение гиперпараметров для GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Инициализация модели
my_tree = MyDecisionTreeRegressor()

# Использование GridSearchCV для поиска лучших гиперпараметров
grid_search = GridSearchCV(estimator=my_tree, param_grid=param_grid,
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Обучение GridSearchCV
grid_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Лучшие параметры:", grid_search.best_params_)

# Прогнозирование с использованием модели с лучшими параметрами
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Оценка модели с лучшими параметрами
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2)



MAE: 16.5
MSE: 320.0
RMSE: 18.0
R²: 0.5



1. MAE: 16.5
   - Средняя ошибка теперь составляет примерно 16. Это существенное улучшение, показывает, что предсказания стали точнее.

2. MSE: 320.0 и RMSE: 18.0
   - Снижение этих значений говорит о том, что модель стала меньше ошибаться как в среднем, так и в случае крупных отклонений.

3. R²: 0.5
   - Хотя коэффициент детерминации немного уменьшился по сравнению с 0.73, модель все еще объясняет 50% вариации в данных, что приемлемо.

### В общем:
После оптимизации модель показывает улучшенные результаты по точности предсказаний, хотя общий уровень объясненной вариации немного снизился. Это указывает на более точное предсказание отдельных значений при возможно более сложной общей структуре данных.
